# graph_generator()


## IMPLEMENTACIÓN

  
Esta función se encarga de archivar las observaciones (.csv) importadas desde Netlogo por fecha y hora, a la vez que interpreta los datos de salida gráficamente. 
El procedimiento para su implementación es el siguiente:

1. Ejecutar el modelo en netlogo (uber.nlogo).
2. Al finalizar las simulaciones, se deberá ejecutar la función graph_generator() desde este notebook.
3. La función guardara las respectivas graficas en directorios clasificados por fecha y hora, con su respectiva transición(gif).



In [ ]:
from PIL import *
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
from apng import APNG
from IPython.display import Image
from os import walk
import time
import shutil

In [20]:
def graph_generator():
    
    #PRIMEROS DATOS EXPORTADOS 
    outs_files = os.listdir('.')
    old_dir = [i for i in outs_files if "." not in i]
    csvs = [i for i in outs_files if ".csv" in i]
    csvs

    folders = [file[:23] for file in csvs]
    folders = set(folders)
    
    #CLASIFICANDOLOS Y MOVIENDOLOS 
    for folder in folders:
        os.makedirs(str(folder+'/people/gif'))
        os.makedirs(str(folder+ '/drivers/gif'))                
        for csv in csvs:
            if folder in csv:
                shutil.move(csv, folder)

    
    listdir1 = os.listdir('.')
    listdir2 = [i for i in listdir1 if "___" in i]
    for n in old_dir:    
        listdir2.remove(str(n))
        listdir2
    listdir2

    for dirr in listdir2:

        outs_files1 = os.listdir('./' + dirr)
        csvs1 = [i for i in outs_files1 if "_.csv" in i]
        csvs1


        for csv1 in csvs1:

            d = pd.read_csv(dirr+'/'+csv1, error_bad_lines=False)
            df = pd.DataFrame(data=d)
            df2 = df[df['breed'] == 'people']
            df3 = df[df['breed'] == 'drivers']

         #GENERANDO GRAFICAS Y GUARDANDOLAS EN DIRECTORIO  
            #people
            plot = sns.jointplot(x="x", y="y", data=df2, kind="kde", size=10, xlim=(-5,40), ylim=(-5,40))
            #import ipdb; ipdb.set_trace()
            plt.savefig(plt.savefig(str(dirr+'/people/people' + '_' + str(csvs1.index(csv1)) + '.png')))
            #drivers 
            sns.jointplot(x="x", y="y", data=df3, kind="kde", size=10, xlim=(-10,50), ylim=(-5,40))
            plt.savefig(plt.savefig(str(dirr+'/drivers/drivers' + '_' + str(csvs1.index(csv1)) + '.png')))

    #ANIMACIÓN

        groups = ['df2','df3']
        for n in groups:
            #people
            if groups[groups.index(n)] == 'df2':

                def ls(ruta = str(dirr+'/people')):

                    return next(walk(ruta))[2]

                images_people = ls()
                images_people


                for m in images_people:                           
                    img = str(str(dirr +'/people/' + m))      
                    img
                    graphs = images_people
                    graphs[int(images_people.index(m))] = img                        

                APNG.from_files(graphs, delay = 400).save(str(dirr+'/people/gif/trans_p_'+dirr+'.png'))
                Image(filename = str(dirr+'/people/gif/trans_p_'+dirr+'.png'))
            #drivers
            elif groups[groups.index(n)] == 'df3':

                def ls(ruta = str(dirr+'/drivers')):

                    return next(walk(ruta))[2]

                images_drivers = ls()
                images_drivers


                for x in images_drivers: 

                    immg = str(str(dirr +'/drivers/' + x))       
                    immg
                    graphs = images_drivers
                    graphs[int(images_drivers.index(x))] = immg                        

                APNG.from_files(graphs,delay=400).save(str(dirr+'/drivers/gif/trans_d_'+dirr+'.png'))
                Image(filename=str(dirr+'/drivers/gif/trans_d_'+dirr+'.png'))



In [21]:
graph_generator()

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/matplotlib/pyplot.py:528: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
